# Task 4
### - Hyperparameter Tunning with Spark and MLlib
#### Group 20: Yuanzhe(Marco) Ma, Matthew Pin, Mark Wang, Tingyu Zhang

### Load libraries

In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
14,application_1618874118876_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Get data

In [2]:
# Get data
aws_credentials = {"key": "", "secret": ""}
## here 100 data points for testing the code
pandas_df = pd.read_csv("s3://mds-s3-student42/output/ml_data_SYD.csv", 
                        storage_options=aws_credentials, index_col=0, parse_dates=True).dropna()
feature_cols = list(pandas_df.drop(columns="Observed").columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Preparing dataset

In [3]:
# Load dataframe and coerce features into a single column called "Features"
# This is a requirement of MLlib
# Here we are converting your pandas dataframe to a spark dataframe, 
# Here "spark" is a spark session I will discuss this in our Wed class.
# read more  here https://blog.knoldus.com/spark-createdataframe-vs-todf/
training = spark.createDataFrame(pandas_df)
assembler = VectorAssembler(inputCols=feature_cols, outputCol="Features")
training = assembler.transform(training).select("Features", "Observed")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Define CV model

In [4]:
# Random forest regressor
rf = RandomForestRegressor(labelCol="Observed", featuresCol="Features")

# Define param grid
rfparamGrid = (ParamGridBuilder()
               .addGrid(rf.numTrees, [10, 50, 100])
               .addGrid(rf.maxDepth, [5, 10])
               .addGrid(rf.bootstrap, [False, True])
               .build())

# Define evaluator
rfevaluator = RegressionEvaluator(labelCol="Observed", metricName="rmse")

# Create 5-fold CrossValidator
rfcv = CrossValidator(estimator = rf,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = rfevaluator,
                      numFolds = 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Perform CV

In [5]:
# Fit model
cvModel = rfcv.fit(training)
print(cvModel)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CrossValidatorModel_26e108169ecc

### Evaluation

In [6]:
print("\nBest model")
print("==========")
print(f"\nCV Score: {min(cvModel.avgMetrics):.2f}")
print(f"numTrees: {cvModel.bestModel.getNumTrees}")
print(f"maxDepth: {cvModel.bestModel.getMaxDepth()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Best model

CV Score: 8.17
numTrees: 100
maxDepth: 5